# Sentiment Analysis dari Instagram comments calon Presiden 2024 sebelum dan sesudah deklrasi calon wakil presiden menggunakan Naive Bayes


## Tujuan : mengetahui perbedaan sentiment dari komentar instagram sebelum dan sesudah deklarasi calon wakil presiden


<ul>
    <li>
    Aurelius Ivan Wijaya (00000054769)
    </li>
    <li>
    Rajendra Abhinaya (00000060445)
    </li>
    <li>
    Maecyntha Irelynn Tantra (00000055038)
    </li>
    <li>
    Patricia theodora (00000054093)
    </li>
<ul>

# Data Collection

apa sih yang sebenernya kita cari?
* web scrapping algoritm untuk data primer (ivan)
* labeling (pat)
* data sekunder (mae)
* stopword library indonesia (abhi)
* cari jurnal referensi yang sudah, sebagai literature review (all, min 4 per person)

# Data Integration

## EDA

# Pre-Proccessing

## Case Folding

## Punctuation & Number & Whitespace Removal

## Stopwords Removal

* this one need research (abhi)

## Stemming / Lemmatization

* this one need research (mae)


## Text Normalization / Noise Removal

* this one need research (ivan)

## Tokenization

* this one need research (mae)

## SMOTE

* this one need research (ivan)

## K-Fold Cross Validation

* this one need research (abhi)

## Workload Distribution?

# Modeling

<!-- * kemungkinan Binomial Naive Bayes -->

# Model Evaluation

* this one need research (pat)